In [1]:
# импортируем библиотеки pandas и numpy
import pandas as pd
import numpy as np

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm.csv', sep=';')
data.head(3)

Customer Lifetime Value  Income  Monthly Premium Auto  \
0             18975.456110   65999                   237   
1              4715.321344       0                    65   
2              5018.885233   54500                    63   

   Months Since Last Claim  Months Since Policy Inception  \
0                        1                             14   
1                       19                             56   
2                       28                             17   

   Number of Open Complaints  Number of Policies  Response  
0                          0                   6         0  
1                          0                   3         0  
2                          0                   6         0

In [3]:
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые
from sklearn.model_selection import train_test_split
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(data.drop('Response', axis=1), 
                                                    data['Response'], 
                                                    test_size=0.3,
                                                    stratify=data['Response'],
                                                    random_state=42)

In [4]:
# импортируем класс StandardScaler, выполняющий стандартизацию
from sklearn.preprocessing import StandardScaler
# создаем модель стандартизации – экземпляр класса StandardScaler
standardscaler = StandardScaler()

In [5]:
# обучаем модель стандартизации, т.е. по каждому признаку 
# в обучающем массиве признаков вычисляем
# среднее значение признака и стандартное 
# отклонение признака для трансформации
standardscaler.fit(X_train)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [6]:
# применяем модель стандартизации к обучающему массиву признаков: из исходного 
# значения признака вычитаем среднее значение признака, вычисленное 
# по ОБУЧАЮЩЕМУ массиву признаков, и результат делим на стандартное 
#отклонение признака, вычисленное по ОБУЧАЮЩЕМУ массиву признаков 
X_train_standardscaled = standardscaler.transform(X_train)

# применяем модель стандартизации к тестовому массиву признаков: из исходного 
# значения признака вычитаем среднее значение признака, вычисленное 
# по ОБУЧАЮЩЕМУ массиву признаков, и результат делим на стандартное 
# отклонение признака, вычисленное по ОБУЧАЮЩЕМУ массиву признаков 
X_test_standardscaled = standardscaler.transform(X_test)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()


In [7]:
# импортируем класс LogisticRegression,
# строящий логистическую регрессию
from sklearn.linear_model import LogisticRegression
# создаем модель логистической регрессии – экземпляр класса LogisticRegression
logreg = LogisticRegression(solver='lbfgs', max_iter=200)
# обучаем модель логистической регрессии, т.е. 
# находим параметры - регрессионные коэффициенты
logreg.fit(X_train_standardscaled, y_train)
# оцениваем качество модели на обучающих данных
print('Правильность на обучающей выборке: {:.3f}'.format(
    logreg.score(X_train_standardscaled, y_train)))
# оцениваем качество модели на тестовых данных
print('Правильность на тестовой выборке: {:.3f}'.format(
    logreg.score(X_test_standardscaled, y_test)))

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.900


In [8]:
# вычисляем спрогнозированные значения зависимой переменной
# для тестового массива признаков
logreg_predvalues = logreg.predict(X_test_standardscaled)
logreg_predvalues[:5]

array([0, 0, 0, 0, 0])

In [9]:
# вычисляем спрогнозированные вероятности классов зависимой переменной
# для тестового массива признаков
logreg_probabilities = logreg.predict_proba(X_test_standardscaled)
logreg_probabilities[:5]

array([[0.91027855, 0.08972145],
       [0.89847518, 0.10152482],
       [0.88300257, 0.11699743],
       [0.90234211, 0.09765789],
       [0.92554507, 0.07445493]])

In [10]:
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler()
minmaxscaler.fit(X_train)
X_train_minmaxscaled = minmaxscaler.transform(X_train)
X_test_minmaxscaled = minmaxscaler.transform(X_test)
logreg = LogisticRegression(solver='lbfgs', max_iter=200)
logreg.fit(X_train_minmaxscaled, y_train)
print('Правильность на обучающей выборке: {:.3f}'.format(
    logreg.score(X_train_minmaxscaled, y_train)))
print('Правильность на тестовой выборке: {:.3f}'.format(
    logreg.score(X_test_minmaxscaled, y_test)))

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.900


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [11]:
from sklearn.preprocessing import RobustScaler
robustscaler = RobustScaler()
robustscaler.fit(X_train)
X_train_robustscaled = robustscaler.transform(X_train)
X_test_robustscaled = robustscaler.transform(X_test)
logreg = LogisticRegression(solver='lbfgs', max_iter=200)
logreg.fit(X_train_robustscaled, y_train)
print('Правильность на обучающей выборке: {:.3f}'.format(
    logreg.score(X_train_robustscaled, y_train)))
print('Правильность на тестовой выборке: {:.3f}'.format(
    logreg.score(X_test_robustscaled, y_test)))

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.900
